[[이유한님] 캐글 코리아 캐글 스터디 커널 커리큘럼](https://kaggle-kr.tistory.com/32)

[Statoil/C-CORE Iceberg Classifier Challenge](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge)

[Keras Model for Beginners (0.210 on LB)+EDA+R&D](https://www.kaggle.com/code/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d)

In [1]:
# This Python 3 enviroment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directiory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))    # 윈도우 사용으로 주석처리

# Any results you write to the current directory are saved as output

This kernel is specifically is for Beginners who want's to experiment building CNN using Keras. By using this kernel, you can expect to get good score and also learn keras. Keras is simple frameworks where we can initialize the model and keep stacking the layers we want. It makes building deep neural networks very easy.  
  
__DeepL 번역__  
이 커널은 특히 케라스를 사용하여 CNN을 구축하는 실험을 하고자 하는 초보자를 위한 커널입니다. 이 커널을 사용하면 좋은 점수를 얻고 케라스도 배울 수 있습니다. Keras는 모델을 초기화하고 원하는 레이어를 계속 쌓을 수 있는 간단한 프레임워크입니다. 이를 통해 심층 신경망을 매우 쉽게 구축할 수 있습니다.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
# plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [3]:
# Load the data.
train = pd.read_json("./input/005_statoil-iceberg-classifier-challenge/train.json")

In [4]:
test = pd.read_json("./input/005_statoil-iceberg-classifier-challenge/test.json")

# Intro about the Data.
Sentinet -1 sat is at about 680 Km above earth. Sending pulses of signals at a particular angle of incidence and then recoding it back. Basically those reflected signals are called backscatter. The data we have been given is backscatter coefficient which is the conventional form of backscatter coefficient given by:

$\sigma o(dB) = \beta o (dB) + 10\log10[\sin(ip)/\sin(ic)]$

where

1. ip=is angle of incidence for a particular pixel
2. 'ic ' is angle of incidence for center of the image
3. K =constant.
We have been given $\sigma o$ directly in the data.

Now coming to the features of $\sigma o$

Basically $\sigma o$ varies with the surface on which the signal is scattered from. For example, for a particular angle of incidence, it varies like:

- WATER........... SETTLEMENTS........ AGRICULTURE........... BARREN........
1.HH: -27.001 ................ 2.70252 ................. -12.7952 ................ -17.25790909

2.HV: -28.035 ................ -20.2665 .................. -21.4471 ................. -20.019

As you can see, the HH component varies a lot but HV doesn't. **I don't have the data for scatter from ship, but being a metal object, it should vary differently as compared to ice object.**

WTF is HH HV?

Ok, so this Sentinal Settalite is equivalent to RISTSAT(an Indian remote sensing Sat) and they only Transmit pings in H polarization, **AND NOT IN V polarization**. Those H-pings gets scattered, objects change their polarization and returns as a mix of H and V. **Since Sentinel has only H-transmitter, return signals are of the form of HH and HV only**. Don't ask why VV is not given(because Sentinel don't have V-ping transmitter).

Now coming to features, for the purpose of this demo code, I am extracting all two bands and taking avg of them as 3rd channel to create a 3-channel RGB equivalent.

# DeepL 번역
센티넷 -1 위성은 지구 상공 약 680km에 있습니다. 특정 입사각으로 신호 펄스를 전송한 다음 다시 수신합니다. 기본적으로 반사된 신호를 후방 산란이라고 합니다. 우리에게 주어진 데이터는 후방 산란 계수이며, 이는 전통적인 형태의 후방 산란 계수입니다:

$\sigma o(dB) = \beta o (dB) + 10\log10[\sin(ip)/\sin(ic)]$

여기서

1. IP=특정 픽셀에 대한 입사각
2. 'ic'는 이미지 중앙의 입사각입니다.
3. K =상수.
데이터에 직접 $\sigma o$가 주어졌습니다.

이제 $\sigma o$의 특징에 대해 알아보겠습니다.

기본적으로 $\sigma o$는 신호가 산란되는 표면에 따라 달라집니다. 예를 들어 특정 입사각의 경우 다음과 같이 달라집니다:

- WATER........... 정착지........ 농업........... BARREN........
1.HH: -27.001 ................ 2.70252 ................. -12.7952 ................ -17.25790909

2.HV: -28.035 ................ -20.2665 .................. -21.4471 ................. -20.019

보시다시피 HH 구성 요소는 많이 변하지만 HV는 그렇지 않습니다. **선박에서 발생하는 산란에 대한 데이터는 없지만, 금속 물체이기 때문에 얼음 물체와 다르게 달라질 수 있습니다.**

WTF가 HH HV인가요?

이 센티널 세탈라이트는 인도의 원격탐사 위성인 RISTSAT에 해당하며, H 편광으로만 핑을 전송하고, **V 편광은 전송하지 않습니다**. 이 H-핑은 산란되어 물체가 편광을 바꾸고 H와 V가 혼합된 형태로 되돌아오는데, **센티널은 H-송신기만 있기 때문에 반사 신호는 HH와 HV의 형태로만 되돌아오게 됩니다**. 왜 VV가 주어지지 않는지 묻지 마세요(Sentinel에는 V-ping 송신기가 없기 때문입니다).

이제 기능에 대해 설명하자면, 이 데모 코드에서는 두 대역을 모두 추출하고 그 평균을 3채널로 취해 3채널 RGB에 해당하는 것을 만들었습니다.

In [5]:
# Generate the training data
# Create 3 bands having HH, HV and avg of both
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis], ((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=1)

https://www.kaggle.com/code/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d